# Maze Solver 2.0

In [24]:
maze = [
    [".", ".", ".", ".", ".", ".", ".", ".", ".", ".", "."],
    [".", ".", "#", "#", "#", "#", "#", ".", ".", ".", "."],
    [".", "#", "#", ".", ".", ".", ".", ".", "#", ".", "."],
    [".", ".", "#", ".", ".", ".", ".", ".", "#", ".", "."],
    ["#", ".", "#", ".", ".", ".", ".", ".", "#", ".", "#"],
    [".", ".", "#", ".", ".", ".", ".", ".", ".", ".", "."],
    [".", "#", "#", "#", "#", ".", ".", "#", ".", "#", "."],
    [".", "#", ".", ".", "F", "#", ".", "#", "#", "#", "."],
    [".", "#", ".", "#", "#", "#", "#", "#", ".", ".", "."],
    [".", "#", ".", "#", ".", ".", ".", ".", ".", "#", "."],
    [".", "#", ".", "#", "#", "#", "#", "#", ".", "#", "."],
    [".", "#", ".", ".", ".", "#", ".", ".", ".", ".", "."],
    [".", "#", "#", "#", ".", "#", ".", "#", "#", ".", "."],
    ["S", "#", ".", ".", ".", ".", ".", ".", ".", "#", "."]
]

import collections
import time
import random
import numpy
from termcolor import colored

import turtle
import math

class Pen(turtle.Turtle):
    def __init__(self, color, size):
        turtle.Turtle.__init__(self)
        self.shape("square")
        self.shapesize(size[0], size[1])
        self.color(color)
        self.penup()
        
class Player(turtle.Turtle):
    def __init__(self, color, size):
        turtle.Turtle.__init__(self)
        self.shape("circle")
        self.shapesize(size[0], size[1])
        self.color(color)
        self.penup()
        self.steps = 0
        self.finish = False
        
class Dot(turtle.Turtle):
    def __init__(self, x, y, color, size):
        turtle.Turtle.__init__(self)
        self.shape("circle")
        self.shapesize(size[0], size[1])
        self.color(color)
        self.penup()
        self.steps = 1
        self.goto(x, y)
    
    def destroy(self):
        self.hideturtle()
        
class Finish(turtle.Turtle):
    def __init__(self, x, y, color, size):
        turtle.Turtle.__init__(self)
        self.shape("circle")
        self.shapesize(size[0], size[1])
        self.color(color)
        self.penup()
        self.goto(x, y)
        
class Queue:
    def __init__(self):
        self.elements = collections.deque()
    
    def empty(self):
        return len(self.elements) == 0 #True or False
    
    def put(self, x):
        self.elements.append(x)
    
    def get(self):
        return self.elements.popleft()

class graph:
    def __init__(self, maze = [
    [".", ".", ".", ".", ".", ".", ".", ".", ".", ".", "."],
    [".", ".", "#", "#", "#", "#", "#", ".", ".", ".", "."],
    [".", "#", "#", ".", ".", ".", ".", ".", "#", ".", "."],
    [".", ".", "#", ".", ".", ".", ".", ".", "#", ".", "."],
    ["#", ".", "#", ".", ".", ".", ".", ".", "#", ".", "#"],
    [".", ".", "#", ".", ".", ".", ".", ".", ".", ".", "."],
    [".", "#", "#", "#", "#", ".", ".", "#", ".", "#", "."],
    [".", "#", ".", ".", "F", "#", ".", "#", "#", "#", "."],
    [".", "#", ".", "#", "#", "#", "#", "#", ".", ".", "."],
    [".", "#", ".", "#", ".", ".", ".", ".", ".", "#", "."],
    [".", "#", ".", "#", "#", "#", "#", "#", ".", "#", "."],
    [".", "#", ".", ".", ".", "#", ".", ".", ".", ".", "."],
    [".", "#", "#", "#", ".", "#", ".", "#", "#", ".", "."],
    ["S", "#", ".", ".", ".", ".", ".", ".", ".", "#", "."]
]):
        self.height = len(maze)
        self.width = len(maze[0])
        self.stats = {}
        self.start = ()
        self.path = {} # make this record the succsesful path
        for y in range(self.height):
            for x in range(self.width):
                self.stats[x, y] = maze[y][x]
                if maze[y][x] == "S":
                    self.start = (x, y)
        
    def print_graph(self, player_x = "not", player_y = "not"):
        player_pos = ()
        for y in range(self.height):
            row = []
            for x in range(self.width):
                if player_x is not "not" and player_x == x and player_y == y:
                    if x + 1 == self.width:
                        row.append(" |P|")
                        
                    else:
                        row.append("|P")
                else:
                    if x + 1 == self.width:
                        row.append("|" + self.stats[x, y] + "|")
                    else:
                        row.append("|" + self.stats[x, y])
            print(*row, sep='')
        print()
        
    def in_bounds(self, id):
        (x, y) = id
        return 0 <= x < self.width and 0 <= y < self.height
    
    def passable(self, id):
        return self.stats[id] is not "#"
    
    def unknown(self, known, pos):
        k = []
        for i in range(len(pos)):
            if pos[i] not in known:
                k.append(pos[i])
        return k
    
    def neighbors(self, id):
        (x, y) = id
        results = [(x+1, y), (x, y-1), (x-1, y), (x, y+1)]
        if (x + y) % 2 == 0: 
            results.reverse()
        results = list(filter(self.in_bounds, results))
        results = list(filter(self.passable, results))
        return results
    
    def true_neighbors(self, id):
        (x, y) = id
        results = [(x+1, y), (x, y-1), (x-1, y), (x, y+1)]
        if (x + y) % 2 == 0: 
            results.reverse()
        results = list(filter(self.in_bounds, results))
        return results
    
    def diagonals(self, id):
        (x, y) = id
        results = [(x+1, y+1), (x-1, y-1), (x-1, y+1), (x+1, y-1)]
        if (x + y) % 2 == 0: 
            results.reverse()
            
        results = list(filter(self.in_bounds, results))
        results = list(filter(self.passable, results))
        
        return results
    
    def print_path(self, path):
        if path != None:
            for y in range(self.height):
                row = []
                for x in range(self.width):
                    if [x, y] in path:
                        if x + 1 == self.width:
                            row.append("|P|")
                        else:
                            row.append("|P")
                    else:
                        if x + 1 == self.width:
                            row.append("|" + self.stats[x, y] + "|")
                        else:
                            row.append("|" + self.stats[x, y])

                print(*row, sep='')
        print()
    
    
    def search_1(self, return_len = False):
        frontier = Queue()
        frontier.put(self.start)

        finish = ()
        self.path[self.start] = [[self.start]]

        while not frontier.empty():
            current = frontier.get()

            for next in self.neighbors(current):
                if next not in self.path:
                    self.path[next] = self.path[current] + [[current[0], current[1]]]
                    frontier.put(next)
                if self.stats[next] == "F":
                        finish = next
                        break
        
        if finish != ():
            if not return_len:
                return self.path[finish][2:]
            else:
                print(len(self.path[finish][2:]))
                return len(self.path[finish][2:])
        else:
            print("Impossible maze")
            self.print_graph()
            
    def make_random_maze(self, w, h):
        l = ['.'] * w
        maze = [l] * h
        
        self.height = h
        self.width = w
        self.stats = {}
        for y in range(self.height):
            for x in range(self.width):
                self.stats[x, y] = maze[y][x]
        self.start = (random.randint(0, w), random.randint(0, h))
        self.stats[self.start] = "S"
        self.path = {}
        
        frontier = Queue()
        frontier.put(self.start)
        mapped = [(self.start)]
        current = self.start
        stack = [(self.start)]
    
        while len(mapped) < w*h:
            for next in self.unknown(mapped, list(self.diagonals(current))):
                mapped.append(next)
                if random.random() < 0.5:
                    self.stats[next] = "#"
                else:
                    self.stats[next] = "."
                    stack.append(next)
                    
            possible = list(self.neighbors(current))
            possible = self.unknown(mapped, possible)
            
            if possible != []:
                current = random.choice(possible)
                if random.random() < 0.1:
                    self.stats[next] = "#"
                    stack.append(next)
                else:
                    self.stats[current] = "."
                    mapped.append(current)
                    stack.append(next)
            else:
                current = stack.pop(0)
            
        self.stats[current] = "F"
        self.stats[(current[0], current[1] - 1)] = "."
        self.stats[self.start] = "S"
        
    def divide_maze(self, width, height):
        if width[1] - width[0] >= 3 and height[1] - height[0] >= 3:
            wall_x = random.randint(width[0] + 1, width[1] - 2)
            wall_y = random.randint(height[0] + 1, height[1] - 2)
            for i in range(height[0], height[1] + 1):
                self.stats[wall_x, i] = "#"
            for i in range(width[0], width[1] + 1):
                self.stats[i, wall_y] = "#"

            self.stats[wall_x, random.randint(height[0], wall_y-1)] = "." 
            self.stats[wall_x, random.randint(wall_y+1, height[1])] = "." 
            self.stats[random.randint(width[0], wall_x-1), wall_y] = "." 
            self.stats[random.randint(wall_x+1, width[1]), wall_y] = "." 
            
            self.divide_maze((width[0], wall_x - 1), (height[0], wall_y - 1))
            self.divide_maze((wall_x + 1, width[1]), (height[0], wall_y - 1))
            self.divide_maze((width[0], wall_x - 1), (wall_y + 1, height[1]))
            self.divide_maze((wall_x + 1, width[1]), (wall_y + 1, height[1]))
            
        elif width[1] - width[0] >= 3 and height[1] - height[0] >= 2:
            wall_x = random.randint(width[0] + 1, width[1] - 2)
            for i in range(height[0], height[1] + 1):
                self.stats[wall_x, i] = "#"
            self.stats[wall_x, random.randint(height[0], height[1])] = "." 
            
            self.divide_maze((width[0], wall_x - 1), (height[0], height[1]))
            self.divide_maze((wall_x + 1, width[1]), (height[0], height[1]))
            
            
        elif width[1] - width[0] >=2 and height[1] - height[0] >= 3:
            wall_y = random.randint(height[0] + 1, height[1] - 2)
            for i in range(width[0], width[1] + 1):
                self.stats[i, wall_y] = "#"
            self.stats[random.randint(width[0], width[1]), wall_y] = "." 
            
            self.divide_maze((width[0], width[1]), (wall_y + 1, height[1]))
            self.divide_maze((width[0], width[1]), (wall_y + 1, height[1]))

            
        
    def make_room_maze(self, w = 35, h = 35):
        l = ['.'] * w
        maze = [l] * h
        
        self.height = h
        self.width = w
        self.stats = {}
        for y in range(self.height):
            for x in range(self.width):
                self.stats[x, y] = maze[y][x]
                
        if random.random() >= 0.5:
            if random.random() >= 0.5:
                self.start = (0, random.randint(1, h - 2))
            else:
                self.start = (w - 1, random.randint(1, h - 2))
        else:
            if random.random() >= 0.5:
                self.start = (random.randint(1, w - 2), 0)
            else:
                self.start = (random.randint(1, w - 2), h - 1)
                
        self.stats[self.start] = "S"
        self.path = {}
        
        frontier = Queue()
        frontier.put(self.start)
        mapped = [(self.start)]
        current = self.start
        stack = [(self.start)]
        
        self.divide_maze([0, w - 1], [0, h - 1])
        ok = False
        while not ok:
            (x, y) = (random.randint(0, w - 1), random.randint(0, h - 1))
            if self.stats[x, y] == ".":
                self.stats[x, y] = "F"
                self.stats[self.start] = "S"
                ok = True
                
    def cellular_automaton(self, w = 35, h = 35):
        self.height = h
        self.width = w
        self.stats = {}
        for y in range(self.height):
            for x in range(self.width):
                if random.random() > 0.4:
                    self.stats[x, y] = "."
                else:
                    self.stats[x, y] = "#"
        for i in range(5):
            next_gen_stats = {}
            #rules
            for y in range(self.height):
                for x in range(self.width):
                    neighbors = self.neighbors((x, y)) + self.diagonals((x, y))
                    if self.stats[x, y] == "#":
                        if len(neighbors) == 3: # 3
                            next_gen_stats[x, y] = "."
                        else:
                            next_gen_stats[x, y] = "#"
                    else:
                        if len(neighbors) >= 1 and len(neighbors) <= 5: # 1-5
                            next_gen_stats[x, y] = "."
                        else:
                            next_gen_stats[x, y] = "#"
                        
            self.stats = {k: v for k, v in next_gen_stats.items()}     
#             self.print_graph()
            
        self.start = (random.randint(0, w - 1), random.randint(0, h - 1))
        self.stats[self.start] = "S"
        self.stats[random.randint(0, w - 1), random.randint(0, h - 1)] = "F"
        
    def primitive(self, w = 35, h = 35):
        self.height = h
        self.width = w
        self.stats = {}
        for y in range(self.height):
            for x in range(self.width):
                self.stats[x, y] = "#"    
        
        self.start = (random.randint(0, w - 1), random.randint(0, h - 1))
       
        
        visited = [self.start]
        came_from = {}
        for next in self.true_neighbors(self.start):
            came_from[next] = self.start
        wall_list = self.true_neighbors(self.start)
        
        def check(id):
            return id not in visited
        s = 0
        while wall_list:
            s += 1
#             print(str(wall_list) + " wall list")
#             print(str(visited) + " visited")
            current = random.choice(wall_list)
            visited.append(current)
            wall_list.remove(current)
#             print(str(current) + " current")
            connects_to = tuple(numpy.subtract(current, tuple(numpy.subtract(came_from[current], current))))
#             print(str(connects_to) + " connects to")
            if connects_to not in visited:
                self.stats[current] = "."
                self.stats[connects_to] = "."
                visited.append(connects_to)
                wall_list = wall_list + list(filter(check, self.true_neighbors(connects_to)))
                for next in list(filter(check, self.true_neighbors(connects_to))):
                    came_from[next] = connects_to
            finish = connects_to
#             if s % 1 == 0 and s < 500:
#             self.print_graph()
        self.stats[self.start] = "S"
        self.stats[finish] = "F"
                
    def row_by_row(self, w = 35, h = 35):
        self.height = h
        self.width = w
        self.stats = {}
        passages = []
        for y in range(self.height):
            for x in range(self.width):
                if y == 0:
                    if random.random() > 0.5:
                        self.stats[x, y] = "."
                        passages.append((x, y))
                    else:
                        self.stats[x, y] = "#"
                else:
                    if (x, y) in passages:
                        self.stats[x, y] = "."
                    else:
                        self.stats[x, y] = "#"
                    
#             print(passages)
            new_passages = []
            for x in range(self.width):
                if (x, y) not in passages:
                    if random.random() > 0.5:
                        new_passages.append((x, y + 1))
#             print(new_passages)
            group = []
            for i in range(self.width):
                if self.stats[i, y] == ".":
                    group.append(passages[0])
                    passages.pop(0)
                    if i + 1 == self.width:
                        passages.append(group)
                else:
                    if len(group) >= 1:
                        passages.append(group)
                    group = []
                    
#             print(str(passages) + " passages")
            for i in range(len(passages)):
                num_of_down = random.randint(1, len(passages[i]))
                for d in range(num_of_down):
                    down = random.choice(passages[i])
                    new_passages.append((down[0], down[1] + 1))
                    passages[i].remove(down)
#             print(new_passages)
            new_passages.sort()
#             print(new_passages)
#             print("ok")
            passages = new_passages
#             print(passages)
        self.start = ()
        while self.start == ():
            self.start = (random.randint(0, w - 1), random.randint(0, h - 1))
            if self.stats[self.start] != ".":
                self.start = ()
        self.stats[self.start] = "S"
        self.stats[random.randint(0, w - 1), random.randint(0, h - 1)] = "F"
            
            
            
    def optimize_maze(self):
        
        frontier = Queue()
        frontier.put(self.start)

        finish = ()
        player_pos = ()
        came_from = {}
        path = {}
        came_from[self.start] = None
        path[self.start] = [[self.start]]

        while not frontier.empty():
            current = frontier.get()

            for next in self.neighbors(current):
                if next not in came_from:
                    path[next] = path[current] + [[current[0], current[1]]]
                    frontier.put(next)
                    came_from[next] = current
                    
        max_key = ""
        max_len = 0
        for key in path:
            cur_len = len(path[key])
            if cur_len > max_len:
                max_key = key
               
        for pos, stat in self.stats.items():
            if stat == "F":
                x = pos

        self.stats[x] = "."
#         value = path[max_key][-1]
        self.stats[max_key[0], max_key[1]] = "F"
    
#     def stats():
        

def printFile(arr, filename):
    file = open(filename,"w+")
    for row in arr:
        row = ""
        for spot in row:
            row += str(spot) + ","
                    
        # row = "".join(map(str, row))
        file.write(str(row) + "\n")  
    file.write("\n\n")
    
# def printFile(arr, filename):
#     with open('filename.txt', 'w') as f:
#         sys.stdout = f # Change the standard output to the file we created.
#         print('This message will be written to a file.')
#         sys.stdout = original_stdout # Reset the standard output to its original value
    
        

def print_to_file(filename, num = 1, graph_type = "make_room_maze", size = (35, 35), minimum = 0):
    file = open(filename,"w+")
        
    if num > 0:
        file.write("Daily Mazes:  \n\n")
        graphs = []
        path_length = {}
        while len(graphs) < num:
            i = len(graphs) + 1
            globals()["maze_" + str(i)] = graph(maze)
            getattr(globals()["maze_" + str(i)], graph_type)(size[0], size[1])
            globals()["maze_" + str(i)].optimize_maze()
            x = globals()["maze_" + str(i)].search_1(True)
            if x >= minimum:
                print(str(x) + " succsesful")
                globals()["maze_" + str(i)].print_graph()
                graphs.append(globals()["maze_" + str(i)])
                path_length[i] = x
            
        for i in range(num):
            print(str(len(path_length)) + " path")
            print(path_length)
            print(min(path_length))
            
            
            file.write("Graph number " + str(i + 1) + "\n\n")
            print(min(path_length, key=path_length.get))
            maze_to_print = graphs[min(path_length, key=path_length.get) - 1]
            path_length.pop(min(path_length, key=path_length.get))
                
            w = size[0]
            h = size[1]
            for y in range(0, h):
                row = []
                for x in range(0, w):
                    if x + 1 == w:
                        row.append("|" + maze_to_print.stats[x, y] + "|")
                    else:
                        row.append("|" + maze_to_print.stats[x, y])
                    
                row = "".join(map(str, row))
                file.write(str(row) + "\n")  
            file.write("\n\n")
            
            
            

class MazeLevel():
    def __init__(self, graph):
        self.graph = graph
        self.sprite_size = 1
        
        self.background_color = "black"
        self.wall_color = "white"
        self.dot_color = "brown"
        self.player_color = "green"
        self.finish_color = "red"
        
        self.window_width = 1500#24 * graph.width * self.sprite_size
        self.window_height = 800#24 * graph.height * self.sprite_size
        self.window = turtle.Screen()
        self.window.bgcolor(self.background_color)
        self.window.title("A Maze")
        self.window.setup(self.window_width + 100, self.window_height + 100)
        self.window.tracer(0)
        
        self.sprite_sizer = (self.window_height / graph.height / graph.height, self.window_width / graph.width / graph.width)
        
        if (self.sprite_sizer[0] > self.sprite_sizer[1]):
            self.dot_sizer = self.sprite_sizer[1]
            self.player_sizer = self.sprite_sizer[0]
        if (self.sprite_sizer[1] > self.sprite_sizer[0]):
            self.dot_sizer = self.sprite_sizer[0]
            self.player_sizer = self.sprite_sizer[1]
        
        self.pen = Pen(self.wall_color, self.sprite_sizer)
        self.player = Player(self.player_color, (self.player_sizer,self.player_sizer))
        
        self.graph = graph
        self.dots = []
        self.finish = []
        self.walls = []
        
    def setup_maze(self):
        if self.sprite_sizer[0] > self.sprite_sizer[1]:
            dot_sizer = self.sprite_sizer[1]
            player_sizer = self.sprite_sizer[0]
        if self.sprite_sizer[1] > self.sprite_sizer[0]:
            dot_sizer = self.sprite_sizer[0]
            player_sizer = self.sprite_sizer[1]
        for y in range(self.graph.height):
            for x in range(self.graph.width):
                character = self.graph.stats[x, y]
                screen_x = round(-((self.window_width - (self.window_width / self.graph.width)) / 2) + (x * self.sprite_size * (self.window_width / self.graph.width)))
                screen_y = round(((self.window_height - (self.window_height / self.graph.height)) / 2) - (y * self.sprite_size * (self.window_height / self.graph.height)))

                if character == "#":
                    self.pen.goto(screen_x, screen_y)
                    self.pen.stamp()

                    self.walls.append((screen_x, screen_y))

                if character == "S":
                    self.player.goto(screen_x, screen_y)

                if character == ".":
                    self.dots.append(Dot(screen_x, screen_y, self.dot_color, (self.dot_sizer, self.dot_sizer)))

                if character == "F":
                    self.finish.append(Finish(screen_x, screen_y, self.finish_color, self.sprite_sizer))

    def playLevel(self):
        self.setup_maze()
        print("Setup done")

        turtle.listen()
        turtle.onkey(self.left, "Left")
        turtle.onkey(self.right, "Right")
        turtle.onkey(self.up, "Up")
        turtle.onkey(self.down, "Down")

        while not self.player.finish:
            for dot in self.dots:
                if self.is_collision(dot):
                    self.player.steps += dot.steps
                    print("Steps: {}".format(self.player.steps))
                    dot.destroy()
                    self.dots.remove(dot)
            if self.is_collision(self.finish[0]):
                self.player.finish = True
                print("You Won")
                print("Total Steps: {}".format(self.player.steps))

            self.window.update()
        self.window.clear()
            
    def up(self):
        move_to_x = round(self.player.xcor())
        move_to_y = round(self.player.ycor() + (self.window_height / self.graph.height))
        
        if not self.wall_collision((move_to_x, move_to_y)) and move_to_y < (self.window_height) / 2:
            self.player.goto(move_to_x, move_to_y)
            
    def down(self):
        move_to_x = round(self.player.xcor())
        move_to_y = round(self.player.ycor() - (self.window_height / self.graph.height))
        
        if not self.wall_collision((move_to_x, move_to_y)) and move_to_y > -(self.window_height) / 2:
            self.player.goto(move_to_x, move_to_y)
            
            
    def left(self):
        move_to_x = round(self.player.xcor() - (self.window_width / self.graph.width))
        move_to_y = round(self.player.ycor())
        
        if not self.wall_collision((move_to_x, move_to_y)) and move_to_x > -(self.window_width) / 2:
            self.player.goto(move_to_x, move_to_y)
            
            
    def right(self):
        move_to_x = round(self.player.xcor() + (self.window_width / self.graph.width))
        move_to_y = round(self.player.ycor())

        if not self.wall_collision((move_to_x, move_to_y)) and move_to_x < (self.window_width) / 2:
            self.player.goto(move_to_x, move_to_y)
            
    def is_collision(self, other):
        a = self.player.xcor()-other.xcor()
        b = self.player.ycor()-other.ycor()
        distance = math.sqrt((a ** 2) + (b ** 2))
        
        if distance < 12:
            return True
        else:
            return False
        
    def wall_collision(self, spot):
        for wall in self.walls:
            a = wall[0] - spot[0]
            b = wall[1] - spot[1]
            distance = math.sqrt((a ** 2) + (b ** 2))

            if distance < 12:
                return True
        return False
    

    
class MazeGame():
    def __init__(self, graph):
        self.graph = graph
        self.sprite_size = 1
        
        self.background_color = "black"
        self.wall_color = "white"
        self.dot_color = "brown"
        self.player_color = "green"
        self.finish_color = "red"
        
        self.window_width = 1500#24 * graph.width * self.sprite_size
        self.window_height = 800#24 * graph.height * self.sprite_size
        self.window = turtle.Screen()
        self.window.bgcolor(self.background_color)
        self.window.title("A Maze")
        self.window.setup(self.window_width + 100, self.window_height + 100)
        self.window.tracer(0)
        
        self.sprite_sizer = (self.window_height / graph.height / graph.height, self.window_width / graph.width / graph.width)
        
        self.graph = graph
        self.dots = []
        self.finish = []
        self.walls = []
        
    def setup_maze(self):
        if self.sprite_sizer[0] > self.sprite_sizer[1]:
            dot_sizer = self.sprite_sizer[1]
            player_sizer = self.sprite_sizer[0]
            
            
def dictToArr(dict,w,h):
    arr = []
    for y in range(h):
        row = []
        for x in range(w):
            val = dict[x, y]
            if val == "#":
                row.append(1)
            else:
                row.append(0)
        arr.append(row)
    return arr
            
g1 = graph(maze)
g1.primitive(10, 5)
# g1.optimize_maze()
g1.print_graph()
arr = dictToArr(g1.stats,10,5)
# game1 = MazeLevel(g1)
# game1.playLevel()

# Tell It What To Do

# print_to_file("ex. graph1", 10, "row_by_row", (50, 76), 120)

# g1 = graph(maze)
# path = g1.search_1()
# g1.print_path(path)
# g1.primitive(20, 20)
# g1.print_graph()
# g1.optimize_maze()
# g1.print_graph()
# path = g1.search_1()
# g1.print_path(path)

# g2 = graph(maze)
# g2.row_by_row(15, 15)
# g2.optimize_maze()
# g2.print_graph()
# path = g2.search_1()
# g2.print_path(path)
# print_to_file("Room Game", 30, "make_room_maze", (4, 7))

# g3 = graph(maze)
# g3.make_random_maze(35, 35)
# g3.optimize_maze()
# path = g3.search_1()
# g3.print_path(path)


|.|.|#|.|#|.|#|.|.|.|
|#|.|#|.|#|.|#|.|#|.|
|.|.|.|.|.|S|.|.|#|.|
|#|#|#|.|#|.|#|.|#|#|
|.|.|.|.|#|.|#|.|.|.|

[0, 0, 1, 0, 1, 0, 1, 0, 0, 0]
[1, 0, 1, 0, 1, 0, 1, 0, 1, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
[1, 1, 1, 0, 1, 0, 1, 0, 1, 1]
[0, 0, 0, 0, 1, 0, 1, 0, 0, 0]


In [40]:
for row in arr:
    print(row)

def printFile(arr, filename):
    file = open(filename,"w+")
    for row in arr:
        rowStr = ""
        for spot in row[:-1]:
            rowStr += str(spot) + ","
        rowStr += str(row[len(row)-1])
        file.write(rowStr + "\n") 
        print(rowStr) 
    file.write("\n\n")
    # file.close()
    
# def printFile(arr, filename):
#     with open('filename.txt', 'w') as f:
#         sys.stdout = f # Change the standard output to the file we created.
#         print('This message will be written to a file.')
#         sys.stdout = original_stdout # Reset the standard output to its original value
    
printFile(arr, "RoombaMap.txt")

[0, 0, 1, 0, 1, 0, 1, 0, 0, 0]
[1, 0, 1, 0, 1, 0, 1, 0, 1, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
[1, 1, 1, 0, 1, 0, 1, 0, 1, 1]
[0, 0, 0, 0, 1, 0, 1, 0, 0, 0]
0,0,1,0,1,0,1,0,0,0
1,0,1,0,1,0,1,0,1,0
0,0,0,0,0,0,0,0,1,0
1,1,1,0,1,0,1,0,1,1
0,0,0,0,1,0,1,0,0,0


9

In [ ]:
import arcade

SCREEN_WIDTH = 800
SCREEN_HEIGHT = 600
sprite_size = [24, 24]

MOVEMENT_SPEED = 5

def on_key_press(self, key, modifiers):
    """Called whenever a key is pressed. """

    if key == arcade.key.UP:
        self.player_sprite.change_y = MOVEMENT_SPEED
    elif key == arcade.key.DOWN:
        self.player_sprite.change_y = -MOVEMENT_SPEED
    elif key == arcade.key.LEFT:
        self.player_sprite.change_x = -MOVEMENT_SPEED
    elif key == arcade.key.RIGHT:
        self.player_sprite.change_x = MOVEMENT_SPEED

def on_key_release(self, key, modifiers):
    """Called when the user releases a key. """

    if key == arcade.key.UP or key == arcade.key.DOWN:
        self.player_sprite.change_y = 0
    elif key == arcade.key.LEFT or key == arcade.key.RIGHT:
        self.player_sprite.change_x = 0

class MyGame(arcade.Window):
    """ Main application class. """

    def __init__(self, width, height, graph):
        super().__init__(width, height)
        self.graph = graph
        
        self.dots = []
        self.walls = []

        arcade.set_background_color(arcade.color.AMAZON)

    def setup(self):
        for y in range(self.graph.height):
            for x in range(self.graph.width):
                character = self.graph.stats[x, y]
                screen_x = x * sprite_size[0] * (SCREEN_WIDTH / self.graph.width)
                screen_y = y * sprite_size[1] * (SCREEN_HEIGHT / self.graph.height)

                if character == "#":
                    self.pen.goto(screen_x, screen_y)
                    self.pen.stamp()

                    self.walls.append((screen_x, screen_y))

                if character == "S":
                    self.player.goto(screen_x, screen_y)

                if character == ".":
                    self.dots.append(screen_x, screen_y)
                    draw_dot(screen_x, screen_y)

                if character == "F":
                    self.finish.append(Finish(screen_x, screen_y, "red", sprite_size))
        pass

    def on_draw(self):
        """ Render the screen. """
        arcade.start_render()
        # Your drawing code goes here

    def update(self, delta_time):
        """ All the logic to move, and the game logic goes here. """
        self.physics_engine.update()
        pass
    
    def draw_dot(x, y):
        arcade.draw_lrtb_rectangle_filled(x + 30, x + 50, y - 100, y - 140, arcade.color.DARK_BROWN)
        
    def draw_finish(x, y):
        arcade.draw_lrtb_rectangle_filled(x + 30, x + 50, y - 100, y - 140, arcade.color.DARK_BROWN) 
        
    def draw_wall(x, y):
        arcade.draw_lrtb_rectangle_filled(x + 30, x + 50, y - 100, y - 140, arcade.color.DARK_BROWN)
        

def main():
    game = MyGame(SCREEN_WIDTH, SCREEN_HEIGHT, g1)
    game.setup()
    arcade.run()


if __name__ == "__main__":
    main()

In [ ]:
def chance(dist, imp):
    return dist * imp
   
def chance2(dist, imp):
    return dist ** imp

c1 = chance(0.2, 0.7)
c2 = chance(0.5, 0.7)
c3 = chance(0.6, 0.7)
c4 = chance(0.8, 0.7)
c5 = chance(0.99, 0.7)
